In [1]:
import numpy as np
import copy

In [7]:
def Sigmoid(x):
    res = 1 / (1 + np.exp(-x))
    return res

def d_Sigmoid(x):
    y = x * (1 - x)
    return y

def ReLU(x):
    x = np.maximum(0, x)
    return x
    
def d_ReLU(x):
    #print('d_ReLU: ', x)
    y=x.copy()
    y[y<=0] = 0
    y[y>0] = 1
    #print('d_ReLU: ', x)
    return y


def Linear(x):
    return x

def d_Linear(x):
    #print('d_Linear: ', x)
    y = np.ones(shape=(x.shape), dtype = x.dtype)
    #print('d_Linear: ', y)
    return y

activations_dict = {
'Sigmoid': [Sigmoid, d_Sigmoid],
'ReLU': [ReLU, d_ReLU], 
'Linear': [Linear, d_Linear]
}


In [8]:
class layer:
    def __init__(s, lr = 0.1, prev_size = 2, my_size=2, activation_type = "Sigmoid", weights = None, bias = None):
        s.lr = lr
        s.size = my_size
        s.prev_size = prev_size
        if (np.all(weights == None)):
            s.weights = np.random.random((prev_size, s.size))
        else:
            s.weights = weights.copy()
            
        if (np.all(bias == None)):
            s.bias = np.random.random((s.size))
        else:
            s.bias = bias.copy()
            
        s.activation_type = activation_type
        funcs = activations_dict.get(activation_type)
        s.activation_f = funcs[0]
        s.d_activation_f = funcs[1]


    def activate(s, x):
        return s.activation_f(x)
        
    def d_activate(s, x):
        return s.d_activation_f(x)  

    def forward(s, inputs):
        s.input = np.asarray(inputs)
        s.neurons = np.dot(s.input, s.weights)
        print('x*w ',s.neurons)
        s.neurons+=s.bias
        print('+ bias ',s.neurons)
        s.neurons_activated = s.activate(s.neurons)
        #print("s.input ", s.input)
        print('s.neurons_activated ',s.neurons_activated)
        return s.neurons_activated

    def forward_nu(s, inputs):       
        input_data = np.asarray(inputs)
        neurons = np.dot(input_data, s.weights)
        neurons += s.bias
        neurons_activated = s.activate(neurons)       
        return neurons_activated

    def backprop(s, layer_error):
        #print('s.neurons_activated ', s.neurons_activated)
        #print('layer_error ', layer_error)
        s.delta = layer_error * s.d_activate(s.neurons_activated)
        #print('s.delta ', s.delta)
        s.prev_layer_error = np.dot(s.delta, s.weights.T)
        #print('s.prev_layer_error ', s.prev_layer_error)
        return s.prev_layer_error
        
    def update_weights(s):
        #print('input in layer ', s.input.T)
        s.input_t = s.input.T
        s.x = s.input_t.reshape((s.input_t.shape[0], 1))
        s.d = s.delta.reshape((1, s.delta.shape[0]))
        s.weights += s.lr * np.dot(s.x, s.d)

        s.bias +=  s.lr * s.delta
        
    def print_info(s):
        print("my size: ", s.size)
        print("w: ", s.weights, "\n")
        print("b: ", s.bias, "\n")


        
        
    def print_pic(s):
        print_size = min(2, s.size)
        print("\n╻...\nv...")
        for i in range(print_size):
            #if (i == print_size/2 and print_size != s.size):
                #print ("--", s.size, "--\t", end='')
            print('O\t', end='')
        print ("--", format(s.size, ' 5d') , "--\t", end='')

    def add_neuron(s):     
        add_w = np.zeros(shape=(s.prev_size, 1), dtype=float) + 0.1
        s.weights = np.concatenate((s.weights, add_w.T))
        s.size+=1

    def get_info(s):
        return s.prev_size, s.size, s.weights, s.bias, s.activation_type, s.lr


class input_layer:
    def __init__(s, size):
        s.size = size
        s.values = np.zeros(shape=(size), dtype = float)

    def print_info(s):
        print("IN LAYER\nsize: ", s.size)
        #print(s.values)
    

    def print_pic(s):
        print_size = min(2, s.size)

        for i in range(print_size):
            #if (i == print_size/2 and print_size != s.size):
                #print ("--", s.size, "--\t", end='')
                
            print("| |\t", end='')
        print("")
        for i in range(print_size):
            #if (i == print_size/2 and print_size != s.size):
                #print ("--", s.size, "--\t", end='')
            print(" v \t", end='')
        print("")
        for i in range(print_size):
            #if (i == print_size/2 and print_size != s.size):
                #print ("--", s.size, "--\t", end='')
            print(' @\t', end='')
        print ("--", format(s.size, ' 5d') , "--\t", end='')

    def forward(s, x):
        return x
        
    def forward_nu(s, x):
        return x

    def get_info(s):
        return s.size
        
        


In [4]:

class plastic_nn:
    def __init__(s):
        s.layers = []
        s.n_of_layers = 0
        s.name = 'noname'
        pass

    def give_name(s, name):
        s.name = name
        
    def set_num_of_layers(s, num):
        s.n_of_layers = num
        
    def deep_copy(s):
        return copy.deepcopy(s)

    def forward(s, data):
        for lay in s.layers:
            data = lay.forward(data)
        s.last_result = data
        return s.last_result
        
    def forward_print(s, data):
        print('in: ',data)
        cnt = 0
        for lay in s.layers:
            data = lay.forward(data)
            print(cnt, ' ', data)
            cnt+=1
        s.last_result = data
        return s.last_result

    def forward_nu(s, data):
        for lay in s.layers:
            data = lay.forward_nu(data)
        last_result = data
        return last_result
        
    
    def backprop(s, correct):
        err = correct-s.last_result
        cnt = 0
        for lay in reversed(s.layers[1:]):
            #print(cnt)
            err = lay.backprop(err)
            cnt+=1

    def backprop_error(s, err):
        #err = correct-s.last_result

        for lay in reversed(s.layers[1:]):
            err = lay.backprop(err)

    def update_w(s):
        i = 0
        for lay in reversed(s.layers[1:]):
            #print('layer idx: ', i)
            i+=1
            lay.update_weights()
            #print('\n')

    def learn_one(s, in_data, target_data):
        pnn.forward(in_data)
        pnn.backprop(target_data)
        pnn.update_w()   
        
    def append_one(s, new_layer, check = False):
        if check and s.n_of_layers!=0:
            last_layer_size = s.layers[-1].size
            if last_layer_size != new_layer.prev_size:
                print("size not match, layer ", s.n_of_layers)
                return
        s.layers.append(new_layer)
        s.n_of_layers+=1
        pass

    def check_layers_sizes(s, check_layers):
        for i in range(1, len(check_layers)):
            if (check_layers[i-1].size != check_layers[i].prev_size):
                print("error between ", i-1, "and ", i)
                return False
        return True
        
    def append(s, new_layers):
        test_layers = np.array([])
        if s.n_of_layers != 0: # if has layers
            test_layers = s.layers[-1] # get last layers
        print(s.n_of_layers)
        test_layers = np.append(test_layers, new_layers) 
                
        if (s.check_layers_sizes(test_layers)):
            for lay in new_layers:
                s.append_one(lay)
            print("added LAYERS succesfully")
            return True
        else:
            print("ERROR adding layers, check info above")
            return False

    def print_info(s):
        print('NAME: ', s.name, ' (', s.n_of_layers, ')')
        for cnt in range(s.n_of_layers):
            print("#", cnt)
            s.layers[cnt].print_info()
            print("")
    
    def print_pic(s):
        print('NAME: ', s.name, ' (', s.n_of_layers, ')')
        cnt = 0
        for lay in s.layers:
            lay.print_pic()
            print("#", cnt, end='')
            cnt+=1
        print("\nOUT |#|\nOUT  v")

    def save(s, file_path):       
        f = open(file_path, "w").close()
        
        f = open(file_path, "a")       
        f.write("{}\n{}\n".format(s.name, s.n_of_layers))       
        input_layer_size = s.layers[0].get_info()
        f.write("{}\n".format(input_layer_size))

        for lay in s.layers[1:]:            
            prev_size, size, weights, bias, activation_type, lr = lay.get_info()
            f.write("{}\n".format(prev_size))
            f.write("{}\n".format(size))
            
            np.savetxt(f, weights)#, fmt='%f')
            np.savetxt(f, bias)#, fmt='%f')
            f.write("{}\n".format(activation_type))
            f.write("{}\n".format(lr))
            
        
        f.close()

        
    def load(s, file_path):
        s.layers = None
        s.layers = []
        s.n_of_layers = 0       
        layers = []
        f = open(file_path, "r")       
        name = f.readline().split()[0]
        total_n_of_layers = int(f.readline().split()[0])
        input_layer_size = int(f.readline().split()[0])

        in_layer = input_layer(input_layer_size)
        layers.append(in_layer)
        
        s.give_name(name)

        for i in range(total_n_of_layers-1):
            prev_size = int(f.readline().split()[0])
            size = int(f.readline().split()[0])
            weights = np.loadtxt(f, max_rows = prev_size)
            bias = np.loadtxt(f, max_rows = size)
            activation_type = f.readline().split()[0]
            lr = float(f.readline().split()[0])

            layers.append(layer(lr = lr, prev_size= prev_size, my_size = size, 
                                activation_type = activation_type, weights = weights, bias = bias))
            
        s.append(layers)             
        f.close()


